## This code computes predicted parameter means and errors using trained compressor and posterior inference networks for CV and test LH sets.

In [1]:
import sys
from pathlib import Path
import numpy as np

import torch
from torch.utils.data import DataLoader

sys.path.append('../')
from utils import datasets
import utils.resnet_cond as resnet
import utils.lightning_flows as LFlows
from utils import lightning_flows_density as LFlowsDensity


/n/home11/aakhmetzhanova/.conda/envs/ccl-env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: aakhmetzhanova. Use `wandb login --relogin` to force relogin


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


### Load trained models

In [4]:
field      = 'Mgas'
home_dir   = Path("/n/netscratch/dvorkin_lab/Lab/aakhmetzhanova/evidence-estimation-Astrid/") 
save_dir   = home_dir / f"trained_models/Maps_{field}/"


# Load summarizer and density estimator.
npe_k_cond = False
n_params   = 6
summary_dim = 40

checkpoint = save_dir / f'summarizer_scale_cond_{field}_Astrid/'
checkpoint = checkpoint / 'k_conv_lr_1e-4/checkpoints/best_val_loss.ckpt'
model      = LFlows.LightningFlowCond.load_from_checkpoint(checkpoint, 
                                                           device=device,
                                                           k_cond='conv',
                                                           npe_k_cond=npe_k_cond,
                                                           summary_dim=summary_dim, n_params=n_params,
                                                           model_name='resnet10t.c3_in1k',)
model.summarizer.eval();
model.density_estimator.eval();

# Smoothing scales
k_min, k_max   = 2., 45
k_smooth_array = np.logspace(np.log10(k_min), np.log10(k_max), num=10)



In [6]:
# Function to compute predicted parameters and error bars 
def params_errors_pred(maps_set, params_set, 
                       maps_mean, maps_std,
                       k_smooth_array,
                       train_frac=0., valid_frac=0., test_frac=1.,
                       seed=1, batch_size=100, verbose=True):
    params_true_all_scales = []
    params_pred_all_scales = []
    errors_pred_all_scales = []

    for i, k_smooth in enumerate(k_smooth_array):
        
        params_true = []
        params_pred = []
        errors_pred = []

        _, _, dset = datasets.create_datasets_maps(maps_set, params_set, 
                                                       train_frac, valid_frac, test_frac, 
                                                       seed=seed, rotations=False, 
                                                       smoothing=True, k_smooth=k_smooth,
                                                       k_min=k_min, k_max=k_max,
                                                       normalize_k=True,
                                                       linear=False, log_scale=True, 
                                                       standardize=True, 
                                                       maps_mean=maps_mean, maps_std=maps_std,)
        dataloader  = DataLoader(dset, batch_size, shuffle = False, )

        for maps_test, params_test, k_test in dataloader:
            with torch.no_grad():
                maps_test    = maps_test.float().to(device=device)
                params_test  = params_test.float().to(device=device)
                k_test       = k_test.float().to(device=device)
                bs           = maps_test.shape[0]

                summaries    = model.summarizer([maps_test, k_test])
                if npe_k_cond:
                    summaries    = torch.cat((summaries, k_test), dim=1)

                params_samples = model.density_estimator(summaries).sample((5_000,))
                params_samples = params_samples.detach().cpu().numpy()
                params_samples = params_samples*(maximum[None, None, :] - minimum[None, None, :])/2 + (minimum[None, None, :]+maximum[None, None, :])/2  
                
                # undo the normalization on the parameters
                params_test = params_test.detach().cpu().numpy()
                params_test = params_test*(maximum[None, :] - minimum[None, :])/2 + (minimum[None, :]+maximum[None, :])/2  
                
                params_true.append(params_test)
                params_pred.append(params_samples.mean(axis=0))
                errors_pred.append(params_samples.std(axis=0))

        params_true = np.concatenate(params_true, axis=0)
        params_pred = np.concatenate(params_pred, axis=0)
        errors_pred = np.concatenate(errors_pred, axis=0)
    
        params_true_all_scales.append(params_true)
        params_pred_all_scales.append(params_pred)
        errors_pred_all_scales.append(errors_pred)
        
    params_true_all_scales = np.array(params_true_all_scales)
    params_pred_all_scales = np.array(params_pred_all_scales)
    errors_pred_all_scales = np.array(errors_pred_all_scales)

    return params_true_all_scales, params_pred_all_scales, errors_pred_all_scales
    
    

## CV set

In [7]:
# Load CV maps for all suites.
minimum = np.array([0.1, 0.6, 0.25, 0.25, 0.5, 0.5])[:n_params] 
maximum = np.array([0.5, 1.0, 4.00, 4.00, 2.0, 2.0])[:n_params] 

splits    = 15
grid      = 256

set_name = 'CV' 
params_S  = np.loadtxt(home_dir / f'data/params_{set_name}_SIMBA.txt',)[:, :n_params] 
params_S  = (params_S - (minimum+maximum)/2)/((maximum - minimum)/2)   # rescale parameters
params_S  = np.repeat(params_S[:, None, :], splits, axis = 1) # reshape the parameters to match the shape of the maps

params_I   = np.loadtxt(home_dir / f'data/params_{set_name}_IllustrisTNG.txt',)[:, :n_params] 
params_I  = (params_I - (minimum+maximum)/2)/((maximum - minimum)/2)   # rescale parameters
params_I  = np.repeat(params_I[:, None, :], splits, axis = 1) # reshape the parameters to match the shape of the maps

params_A   = np.loadtxt(home_dir / f'data/params_{set_name}_Astrid.txt',)[:, :n_params]
params_A  = (params_A - (minimum+maximum)/2)/((maximum - minimum)/2)   # rescale parameters
params_A  = np.repeat(params_A[:, None, :], splits, axis = 1) # reshape the parameters to match the shape of the maps


dset_size = params_S.shape[0]
maps_S    = np.load(home_dir / f'data/2D/Maps_{field}_SIMBA_{set_name}_z=0.00.npy').reshape(dset_size, -1, 1, grid, grid)[:, :splits]
maps_I    = np.load(home_dir / f'data/2D/Maps_{field}_IllustrisTNG_{set_name}_z=0.00.npy').reshape(dset_size, -1, 1, grid, grid)[:, :splits] #+'data/3D/Pk_m_IllustrisTNG_CV.npy')[:, k_idx, 1]
maps_A    = np.load(home_dir / f'data/2D/Maps_{field}_Astrid_{set_name}_z=0.00.npy').reshape(dset_size, -1, 1, grid, grid)[:, :splits] #+'data/3D/Pk_m_Astrid_CV.npy')[:, k_idx, 1]

# Normalization constants
maps_LH    = np.load(home_dir  / f'data/2D/Maps_{field}_Astrid_LH_z=0.00.npy').reshape(1000, -1, 1, grid, grid)[:, :splits]
maps_mean, maps_std = np.log10(maps_LH).mean(), np.log10(maps_LH).std()



In [ ]:
params_true_I, params_pred_I, errors_pred_I = params_errors_pred(maps_I, params_I, 
                                                  maps_mean, maps_std,
                                                  k_smooth_array,)
params_true_S, params_pred_S, errors_pred_S = params_errors_pred(maps_S, params_S, 
                                                  maps_mean, maps_std,
                                                  k_smooth_array,)
params_true_A, params_pred_A, errors_pred_A = params_errors_pred(maps_A, params_A, 
                                                  maps_mean, maps_std,
                                                  k_smooth_array,)

results_dir = Path("results/")

np.save(results_dir / f'{field}_params_IllustrisTNG_CV.npy', 
       [params_true_I, params_pred_I, errors_pred_I])
np.save(results_dir / f'{field}_params_SIMBA_CV.npy', 
       [params_true_S, params_pred_S, errors_pred_S])
np.save(results_dir / f'{field}_params_Astrid_CV.npy', 
       [params_true_A, params_pred_A, errors_pred_A])

## LH test set

In [9]:
# load all LH maps
minimum = np.array([0.1, 0.6, 0.25, 0.25, 0.5, 0.5])[:n_params] 
maximum = np.array([0.5, 1.0, 4.00, 4.00, 2.0, 2.0])[:n_params] 

splits    = 15
grid      = 256

set_name = 'LH' 
params_S  = np.loadtxt(home_dir / f'data/params_{set_name}_SIMBA.txt',)[:, :n_params] 
params_S  = (params_S - (minimum+maximum)/2)/((maximum - minimum)/2)   # rescale parameters
params_S  = np.repeat(params_S[:, None, :], splits, axis = 1) # reshape the parameters to match the shape of the maps

params_I   = np.loadtxt(home_dir / f'data/params_{set_name}_IllustrisTNG.txt',)[:, :n_params] 
params_I  = (params_I - (minimum+maximum)/2)/((maximum - minimum)/2)   # rescale parameters
params_I  = np.repeat(params_I[:, None, :], splits, axis = 1) # reshape the parameters to match the shape of the maps

params_A   = np.loadtxt(home_dir / f'data/params_{set_name}_Astrid.txt',)[:, :n_params]
params_A  = (params_A - (minimum+maximum)/2)/((maximum - minimum)/2)   # rescale parameters
params_A  = np.repeat(params_A[:, None, :], splits, axis = 1) # reshape the parameters to match the shape of the maps


dset_size = params_S.shape[0]
maps_S    = np.load(home_dir / f'data/2D/Maps_{field}_SIMBA_{set_name}_z=0.00.npy').reshape(dset_size, -1, 1, grid, grid)[:, :splits]
maps_I    = np.load(home_dir / f'data/2D/Maps_{field}_IllustrisTNG_{set_name}_z=0.00.npy').reshape(dset_size, -1, 1, grid, grid)[:, :splits] #+'data/3D/Pk_m_IllustrisTNG_CV.npy')[:, k_idx, 1]
maps_A    = np.load(home_dir / f'data/2D/Maps_{field}_Astrid_{set_name}_z=0.00.npy').reshape(dset_size, -1, 1, grid, grid)[:, :splits] #+'data/3D/Pk_m_Astrid_CV.npy')[:, k_idx, 1]

# Normalization constants
maps_LH    = np.load(home_dir  / f'data/2D/Maps_{field}_Astrid_LH_z=0.00.npy').reshape(1000, -1, 1, grid, grid)[:, :splits]
maps_mean, maps_std = np.log10(maps_LH).mean(), np.log10(maps_LH).std()



In [ ]:
train_frac, valid_frac, test_frac = 0.9, 0.05, 0.05

params_true_I, params_pred_I, errors_pred_I = params_errors_pred(maps_I, params_I, 
                                                  maps_mean, maps_std,
                                                  k_smooth_array,
                                                  train_frac, valid_frac, test_frac)
params_true_S, params_pred_S, errors_pred_S = params_errors_pred(maps_S, params_S, 
                                                  maps_mean, maps_std,
                                                  k_smooth_array,
                                                  train_frac, valid_frac, test_frac)
params_true_A, params_pred_A, errors_pred_A = params_errors_pred(maps_A, params_A, 
                                                  maps_mean, maps_std,
                                                  k_smooth_array,
                                                  train_frac, valid_frac, test_frac)
results_dir = Path("results/")

np.save(results_dir / f'{field}_params_IllustrisTNG_LH_test.npy', 
       [params_true_I, params_pred_I, errors_pred_I])
np.save(results_dir / f'{field}_params_SIMBA_LH_test.npy', 
       [params_true_S, params_pred_S, errors_pred_S])
np.save(results_dir / f'{field}_params_Astrid_LH_test.npy', 
       [params_true_A, params_pred_A, errors_pred_A])